In [1]:
import json
import pandas as pd
import os
import glob
from tqdm import tqdm
import re
from pandas.io.json import json_normalize
from datetime import datetime
import numpy as np

In [2]:
adam = pd.read_csv('adam_run.csv')
pete = pd.read_csv('pete_run.csv')
jason = pd.read_csv('jason_run.csv')
jeremy = pd.read_csv('jeremy_run.csv')

In [3]:
all_df = pd.concat([adam, pete, jason, jeremy], axis = 0)
all_df 

,Unnamed: 0,activity_id,altitude,cadence,distance,grade_adjusted_distance,grade_smooth,time,date,lat,long,runner
0,0.0,494499582.0,4.4,0.0,0.000000,0.0,4.5,0.000000,2/15/2016,33.656636,-118.002546,adam
1,1.0,494499582.0,5.6,60.0,0.003045,5.4,2.5,0.083333,2/15/2016,33.656671,-118.002513,adam
2,2.0,494499582.0,5.6,78.0,0.016404,28.5,1.7,0.183333,2/15/2016,33.656823,-118.002371,adam
3,3.0,494499582.0,5.6,79.0,0.029577,49.7,0.0,0.283333,2/15/2016,33.656978,-118.002238,adam
4,4.0,494499582.0,5.6,78.0,0.042750,72.2,1.4,0.400000,2/15/2016,33.657159,-118.002165,adam
5,5.0,494499582.0,5.6,78.0,0.055923,93.4,0.0,0.516667,2/15/2016,33.657331,-118.002078,adam
6,6.0,494499582.0,6.6,78.0,0.059776,99.8,0.7,0.550000,2/15/2016,33.657381,-118.002109,adam
7,7.0,494499582.0,5.6,80.0,0.065865,109.7,0.3,0.600000,2/15/2016,33.657441,-118.002186,adam
8,8.0,494499582.0,6.0,80.0,0.078479,129.4,-0.8,0.716667,2/15/2016,33.657560,-118.002351,adam
9,9.0,494499582.0,5.8,81.0,0.091590,150.9,0.5,0.833333,2/15/2016,33.657680,-118.002528,adam


In [25]:
#group by keys, get aggregrate of stats columns
g = all_df.groupby(['activity_id', 'date']).agg(['mean', 'max'])[['distance', 'time']].reset_index()

#flatten the columns
g.columns = g.columns.map('_'.join)
g

,activity_id_,date_,distance_mean,distance_max,time_mean,time_max
0,494499582.0,2/15/2016,28.936226,55.623099,396.463839,790.700000
1,495086811.0,2/16/2016,0.859900,1.627309,12.891042,26.750000
2,495293794.0,2/16/2016,8.643894,17.518753,173.695024,312.466667
3,495928847.0,2/17/2016,18.303282,36.345617,312.518371,620.350000
4,496088320.0,2/17/2016,11.223892,22.231481,252.316818,473.766667
5,496741568.0,2/18/2016,17.783836,36.263596,374.449941,732.616667
6,497471627.0,2/19/2016,3.278050,7.156518,99.503520,165.950000
7,497492517.0,2/19/2016,20.652857,41.477335,378.896181,809.333333
8,498361372.0,2/20/2016,8.047991,15.623012,187.931431,345.166667
9,498362296.0,2/20/2016,13.237150,24.551059,319.378298,591.600000


In [23]:
# calculate pace
g['pace'] = g.time_mean / g.distance_mean
g

,activity_id_,date_,distance_mean,distance_max,time_mean,time_max,pace
0,527720644,3/26/2016,26.471489,50.260230,411.808625,810.450000,15.556685
1,528605973,3/27/2016,25.238492,50.014851,377.220373,752.666667,14.946233
2,529918577,3/28/2016,25.481618,50.004536,367.129498,720.000000,14.407621
3,531336926,3/29/2016,6.918260,14.001543,78.978511,178.966667,11.415950
4,531337220,3/29/2016,1.442248,2.819534,21.265115,51.833333,14.744428
5,531341127,3/30/2016,27.709049,55.008003,509.900356,992.983333,18.401944
6,533076747,3/31/2016,24.435139,50.096872,396.569981,810.133333,16.229496
7,533080392,4/1/2016,24.766190,50.028335,367.075556,792.766667,14.821640
8,534048784,4/2/2016,25.783016,51.295683,389.951371,786.383333,15.124351
9,535234982,4/3/2016,25.611625,51.314262,392.969213,793.000000,15.343392


In [4]:
all_df.to_csv("combined_runners.csv")
